In [ ]:
# Clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install dependencies
!pip install -r requirements.txt


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!python detect.py --weights yolov5s.pt --source PXL_20250424_183749893.mp4 --conf 0.4 --save-txt --save-conf


In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = 'runs/detect/exp/ChikFil A.mp4'  # Update if filename differs

mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f'<video width=640 controls><source src="{data_url}" type="video/mp4"></video>')


In [ ]:
import os

# Config
label_folder = '/content/yolov5/runs/detect/exp/labels'
base_filename = 'PXL_20250424_183749893'

def inspect_boxes(frame_number):
    label_file = os.path.join(label_folder, f"{base_filename}_{frame_number}.txt")

    if not os.path.exists(label_file):
        print(f"Label file not found for frame {frame_number}: {label_file}")
        return

    print(f"\nInspecting bounding boxes for frame {frame_number}:\n")
    with open(label_file, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        parts = list(map(float, line.strip().split()))
        class_id, x_c, y_c, bw, bh = parts[:5]

        if int(class_id) == 0:
            area = bw * bh
            print(f"Person {i+1}: y_center = {y_c:.3f}, box_area = {area:.4f}")
        else:
            print(f"Object {i+1} (class {int(class_id)}): Skipped")

# 🔍 Example: Check frame 1
inspect_boxes(30)


In [ ]:
import os

label_path = '/content/yolov5/runs/detect/exp/labels'
base_filename = 'PXL_20250424_183749893'
frame_counts = []

# Loop through all label files
for frame_number in range(1880):  # or len(os.listdir(label_path)) if uncertain
    label_file = os.path.join(label_path, f"{base_filename}_{frame_number}.txt")

    count = 0

    if not os.path.exists(label_file):
        frame_counts.append(0)  # No detections = 0 people
        continue

    with open(label_file, 'r') as f:
        lines = f.readlines()

        for line in lines:
            parts = list(map(float, line.strip().split()))
            class_id, x_c, y_c, bw, bh = parts[:5]
            box_area = bw * bh

            # Filter: Only count customers (large boxes, low enough in frame)
            if int(class_id) == 0 and box_area > 0.03:
                count += 1

    frame_counts.append(count)

# Preview
print("Filtered customer counts (first 30 frames):", frame_counts[:30])


In [ ]:
import cv2
from tqdm import tqdm

# Input video
input_video = 'PXL_20250424_183749893.mp4'
cap = cv2.VideoCapture(input_video)

# Output video settings
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = 'output_wait_time_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_num = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop through each frame and add wait time text
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_num < len(wait_times):
        wait = wait_times[frame_num]
    else:
        wait = 0

    # Draw wait time text
    text = f"Estimated Wait Time: {wait} min"
    cv2.putText(frame, text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    # Optional: draw people count too
    count = frame_counts[frame_num] if frame_num < len(frame_counts) else 0
    cv2.putText(frame, f"People in Queue: {count}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

    out.write(frame)
    frame_num += 1

cap.release()
out.release()
print(f"Video saved to {output_path}")


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output_wait_time_video.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f'<video width=640 controls><source src="{data_url}" type="video/mp4"></video>')


In [ ]:
import matplotlib.pyplot as plt

# Make sure frame_counts and wait_times are already defined

plt.figure(figsize=(14, 5))

# Plot people in queue
plt.plot(frame_counts, label='People in Queue', color='blue', linewidth=2)

# Plot wait time
plt.plot(wait_times, label='Estimated Wait Time (min)', color='green', linewidth=2)

# Formatting
plt.title("Queue Length vs Estimated Wait Time")
plt.xlabel("Frame Number")
plt.ylabel("Count / Minutes")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import cv2
import os
from tqdm import tqdm

# Config paths
input_video = 'PXL_20250424_183749893.mp4'
labels_dir = '/content/yolov5/runs/detect/exp/labels'
base_filename = 'PXL_20250424_183749893'

cap = cv2.VideoCapture(input_video)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

output_path = 'output_wait_time_with_boxes.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    label_file = os.path.join(labels_dir, f"{base_filename}_{frame_num}.txt")

    count = 0  # customer counter

    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            lines = f.readlines()

            for line in lines:
                parts = list(map(float, line.strip().split()))
                class_id, x_c, y_c, bw, bh = parts[:5]
                box_area = bw * bh

                if int(class_id) == 0 and box_area > 0.03:
                    # Convert to pixel coordinates
                    x_c, y_c, bw, bh = x_c * frame_width, y_c * frame_height, bw * frame_width, bh * frame_height
                    x1 = int(x_c - bw / 2)
                    y1 = int(y_c - bh / 2)
                    x2 = int(x_c + bw / 2)
                    y2 = int(y_c + bh / 2)

                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    count += 1

    # Determine wait time
    wait = count * 2  # 2 minutes per person

    # Draw wait time
    cv2.putText(frame, f"Estimated Wait Time: {wait} min", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    # Draw people count
    cv2.putText(frame, f"People in Queue: {count}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

    # Write frame
    out.write(frame)
    frame_num += 1

cap.release()
out.release()
print(f"Video saved to {output_path}")


In [ ]:
import cv2
import os
import zipfile

# Paths
video_path = 'output_wait_time_with_boxes.mp4'  # Final video
output_folder = '/content/extracted_frames_350'
zip_filename = '/content/extracted_frames_350.zip'

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)

frame_num = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save only every 350th frame
    if frame_num % 350 == 0:
        frame_filename = os.path.join(output_folder, f"frame_{frame_num:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        saved_count += 1

    frame_num += 1

cap.release()
print(f"Saved {saved_count} frames at every 350 frames to {output_folder}")

# Zip the folder
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)

print(f"Zipped images to {zip_filename}")


In [ ]:
import cv2

cap = cv2.VideoCapture('output_wait_time_with_boxes.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

print(f"Total frames in the video: {total_frames}")
